# Libraries

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import pickle
import random
import time
import os

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

import lightgbm as lgb
from sklearn.neural_network import MLPRegressor

from scipy.optimize import minimize

import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_rows = 120
pd.options.display.max_columns = 100

import warnings
warnings.simplefilter('ignore')

# Parameters

In [ ]:
N_SPLITS = 10
N_ESTIMATORS = 10000
EARLY_STOPPING_ROUNDS = 200
VERBOSE = 1000
SEED = 299792458

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(SEED)

# Datasets

In [ ]:
INPUT = Path("../input/tabular-playground-series-aug-2021")
INPUT_PRED = Path("../input/tps08-pred")

train = pd.read_csv(INPUT / "train.csv")
train['pred'] = np.load(INPUT_PRED / "oof.npy")

test = pd.read_csv(INPUT / "test.csv")
test['pred'] = np.load(INPUT_PRED / "pred.npy")

submission = pd.read_csv(INPUT / "sample_submission.csv")

scale_features = [col for col in test.columns if 'f' in col]
features = scale_features + ['pred']
target = 'loss'

In [ ]:
ss = StandardScaler()
train[scale_features] = ss.fit_transform(train[scale_features])
test[scale_features] = ss.transform(test[scale_features])

In [ ]:
train.shape, test.shape

# LightGBM

In [ ]:
lgb_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'n_estimators': N_ESTIMATORS,
    'random_state': SEED,
    'learning_rate': 5e-3,
    'subsample': 0.8,
    'subsample_freq': 1,
    'colsample_bytree': 0.6,
    'reg_alpha': 6.4,
    'reg_lambda': 1.8,
    'min_child_weight': 256,
    'min_child_samples': 20,
    'importance_type': 'gain',
}

In [ ]:
mlp_oof = np.zeros(train.shape[0])
mlp_pred = np.zeros(test.shape[0])

lgb_oof = np.zeros(train.shape[0])
lgb_pred = np.zeros(test.shape[0])
feature_importances = pd.DataFrame()

kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

for fold, (trn_idx, val_idx) in enumerate(kf.split(X=train[features])):
    print(f"===== fold {fold} =====")
    X_train, y_train = train[features].iloc[trn_idx], train[target].iloc[trn_idx]
    X_valid, y_valid = train[features].iloc[val_idx], train[target].iloc[val_idx]
    X_test = test[features]

    start = time.time()
    model = MLPRegressor(hidden_layer_sizes=50,
                         early_stopping=True,
                         n_iter_no_change=100,
                         solver='adam',
                         shuffle=True,
                         random_state=SEED)
    model.fit(X_train,y_train)

    mlp_oof[val_idx] = model.predict(X_valid)
    mlp_pred += model.predict(X_test) / N_SPLITS
    
    elapsed = time.time() - start
    
    rmse = mean_squared_error(y_valid, mlp_oof[val_idx], squared=False)
    print(f"fold {fold} - mlp rmse: {rmse:.6f}, elapsed time: {elapsed:.2f}sec\n")
    
    start = time.time()
    model = lgb.LGBMRegressor(**lgb_params)
    model.fit(
        X_train, 
        y_train,
        eval_set=[(X_valid, y_valid)],
        eval_metric='rmse',
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
        verbose=VERBOSE,
    )

    fi_tmp = pd.DataFrame()
    fi_tmp['feature'] = model.feature_name_
    fi_tmp['importance'] = model.feature_importances_
    fi_tmp['fold'] = fold
    fi_tmp['seed'] = SEED
    feature_importances = feature_importances.append(fi_tmp)
    
    lgb_oof[val_idx] = model.predict(X_valid)
    lgb_pred += model.predict(X_test) / N_SPLITS
    
    elapsed = time.time() - start
    
    rmse = mean_squared_error(y_valid, lgb_oof[val_idx], squared=False)
    print(f"fold {fold} - lgb rmse: {rmse:.6f}, elapsed time: {elapsed:.2f}sec\n")

rmse = mean_squared_error(train[target], mlp_oof, squared=False)
print(f"oof mlp rmse = {rmse:.6f}")

rmse = mean_squared_error(train[target], lgb_oof, squared=False)
print(f"oof lgb rmse = {rmse:.6f}")

np.save("mlp_oof.npy", mlp_oof)
np.save("mlp_pred.npy", mlp_pred)

np.save("lgb_oof.npy", lgb_oof)
np.save("lgb_pred.npy", lgb_pred)

# Ensemble

In [ ]:
def class_optimizer(X, a0, a1):
    oof = X[0]*a0 + (1-X[0])*a1
    return mean_squared_error(train[target], oof, squared=False)

res = minimize(
    fun=class_optimizer,
    x0=[0.2],
    args=tuple([mlp_oof, lgb_oof]),
    method='Nelder-Mead',
    options={'maxiter': 300})

print(res)
print(f"coef0 {res.x[0]}, coef1 {1-res.x[0]}")

In [ ]:
ensemble_oof = res.x[0] * mlp_oof + (1-res.x[0]) * lgb_oof
ensemble_pred = res.x[0] * mlp_pred + (1-res.x[0]) * lgb_pred

print(mean_squared_error(train[target], ensemble_oof, squared=False))

# Submission

In [ ]:
submission['loss'] = ensemble_pred
submission.to_csv("submission.csv", index=False)

submission